playlist_processing.ipynb

cleaned_song_meta.json(음악 메타 데이터), train.json(플레이리스트)를 사용해
플레이리스트에 존재하는 태그들을 song_id에 매핑한다.이 때 태그는 가장 많이 나온 10개의 태그만 저장한다.

count는 플레이리스트에 얼마나 많이 등장했는지 weight를 주기 위해 저장한다.

In [41]:
import json
import pandas as pd

"""
Method
"""

# Load cleaned song meta data(79,814)
def load_song_meta() -> pd.DataFrame:
    with open('./cleaned_song_meta.json', 'r', encoding='UTF8') as f:
        data = json.load(f)
    df = pd.json_normalize(data)
    df = df[['id', 'song_name', 'artist_name_basket']]
    df['tags'] = [[] for _ in range(len(df))]
    df['count'] = 0
    return df

# Load playlist data(115,071)
def load_playlist() -> pd.DataFrame:
    with open('./train.json', 'r', encoding='UTF8') as f:
        data = json.load(f)
    df = pd.json_normalize(data)
    df.sort_values(by=['like_cnt', 'updt_date'], ascending=False, inplace=True)
    return df

# Add playlist's @tags to each songs and update @count to give weights.
def processing_playlist(song_df: pd.DataFrame, playlist_df: pd.DataFrame) -> pd.DataFrame:
    for idx_, row in playlist_df.iterrows():
        cnt = row.like_cnt
        song_list = row.songs
        tags = row.tags
        for song in song_list:
            if song in song_df['id'].values:
                idx, = song_df.index[song_df['id'] == song].values
                song_df.at[idx, 'count'] += 1
                origin_tags = dict(song_df.at[idx, 'tags'])
                
                for tag in tags:
                    if tag not in origin_tags: origin_tags[tag] = 1
                    elif tag in origin_tags: origin_tags[tag] += 1
                
                if len(origin_tags) > 10:
                    new_dict = {}
                    for tag, cnt in sorted(list(origin_tags.items()), key=lambda x: -x[1])[:10]:
                        new_dict[tag] = cnt
                    song_df.at[idx, 'tags'] = list(new_dict.items())
                else:
                    song_df.at[idx, 'tags'] = list(origin_tags.items())
    
    for idx, row in song_df.iterrows():    
        song_df.at[idx, 'tags'] = list(dict(row.tags).keys())
    
    filter = song_df.tags.apply(lambda x: len(x) >= 10)
    song_df = song_df[filter]
    
    for idx, row in song_df.iterrows():
        song_df.at[idx, 'tags'] = ' '.join(song_df.at[idx, 'tags'])
    
    song_df = song_df[song_df['count'] >= 10]
        
    song_df.sort_values(by='count', ascending=False, inplace=True)
    return song_df
        

In [42]:
processed_song = processing_playlist(load_song_meta(), load_playlist())

AttributeError: 'Series' object has no attribute 'tags'

In [44]:
for idx, row in processed_song.iterrows():
    print(type(processed_song.at[idx, 'tags']))

['감성', '새벽', '밤', '인디', '새벽감성', '여름밤', '잠들기전', '밤에듣기좋은노래', '새벽에듣기좋은노래', '공부']
['새벽', '감성', '밤', '힐링', '기분전환', '가을', '카페', '잔잔', '가을감성', '노래추천']
['발라드', '비오는날', '비', '밤', '혼자', '장마', '감성발라드', '편안한', '센치한', '감성적인']
['새벽', '인디', '밤', '감성', '새벽감성', '잠들기전', '여름밤', '밤에듣기좋은노래', '새벽에듣기좋은노래', '공부']
['기분전환', '힐링', '카페', '발라드', '새벽', '밤', '기분좋은', '인디음악', '잔잔', '노래추천']
['발라드', '휴식', '인디', '어쿠스틱', '커피', '여행', '솔로', '라떼', '기타', '윤딴딴']
['발라드', '추억', '회상', '노래방', '오후', '저녁', '센치', '노을', '2000년대', '싸이월드']
['감성', '인디', '분위기', '혼자', '비오는날', '비', '흐린날', '센치한', '감성적인', '매니아']
['감성', '추억', '새벽', '이별', '그리움', '눈물', '아련한', '후회', '보고픔', '짝사랑할때듣는노래']
['사랑', '설렘', '힐링', '봄', '카페', '밤', '달달', '가을', '인디음악', '명곡']
['감성', '추억', '새벽', '이별', '슬픔', '그리움', '눈물', '힘들때', '아련한', '후회']
['드라이브', '잔잔한', '가을', '밤', '분위기', '새벽감성', 'RnB', '겨울밤', '밤에듣기좋은노래', '하루']
['비오는날', '발라드', '비', '저녁', '장마', '감성발라드', '혼자', '센치한', '감성적인', '편안한']
['휴식', '카페', '기분전환', '커피', '겨울', '편안한', '스트레스', '주말', '여행', '방안에서']
['설렘', '발라드', '휴식', '힐링', '감성'

KeyboardInterrupt: 

In [29]:
processed_song.to_json('processed_song.json', orient='records')